In [8]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools

In [12]:
#Search term to be used on twitter scrape
search = "@McDonalds"

#how to scrape
scraped_tweets= sntwitter.TwitterSearchScraper(search).get_items()
sliced_scraped_tweets = itertools.islice(scraped_tweets,10)

#create df of all the new data
df = pd.DataFrame(sliced_scraped_tweets)[['date', 'content']]

In [13]:
df

,date,content
0,2021-07-13 13:17:19+00:00,@HoldenPro @ProWilldy @McDonalds Mmmmmm caffei...
1,2021-07-13 13:17:14+00:00,From embarking on long journeys ✈️ and in need...
2,2021-07-13 13:17:08+00:00,@CheckersRallys @KrisAndersonTV @BurgerKing @M...
3,2021-07-13 13:16:55+00:00,I love McDonalds French fries and fish sandwic...
4,2021-07-13 13:15:56+00:00,@KrisAndersonTV @BurgerKing @McDonalds @Chickf...
5,2021-07-13 13:15:03+00:00,@crockettonair @foodbeast @McDonalds @Chickfil...
6,2021-07-13 13:14:31+00:00,#faistonbeurre #fashions #hype #hope #undergro...
7,2021-07-13 13:13:55+00:00,@kwilesjrphoto @KrisAndersonTV @BurgerKing @Mc...
8,2021-07-13 13:13:17+00:00,I live at McDonald’s and by that I mean I live...
9,2021-07-13 13:12:26+00:00,@Bron1954 @drunkvegans @BudderedUp @Aivan10100...


In [14]:
#create array that will store the names of all the cities
city_array = []

#import dataframe containing populations of cities in california
california_cities_df = pd.read_csv('cal_populations_city.csv')


In [15]:
california_cities_df['pop_april_2010'].count()

455

In [16]:
california_cities_df.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
0,San Bernardino,Adelanto,1970,2164.0,8517,18130,31765
1,Los Angeles,Agoura Hills,1982,NaN,20390,20537,20330
2,Alameda,Alameda,1854,63852.0,76459,72259,73812
3,Alameda,Albany,1908,15130.0,16327,16444,18539
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089


In [17]:
#only want to look at cities bigger than 50k to ensure there is enough data
bigger_ca_cities_df = california_cities_df[california_cities_df['pop_april_2010']>50000] 

#rslt_df = dataframe[dataframe['Percentage'] > 70] 

In [18]:
bigger_ca_cities_df.count()

#162 cities meet the criteria

County                 162
City                   162
Incorportation_date    162
pop_april_1980         148
pop_april_1990         162
pop_april_2000         162
pop_april_2010         162
dtype: int64

In [19]:
#df of bigger cities in CA
bigger_ca_cities_df.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
2,Alameda,Alameda,1854,63852.0,76459,72259,73812
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089
7,Orange,Anaheim,1876,219494.0,266406,328014,336265
10,Contra Costa,Antioch,1872,42683.0,62195,90532,102372
11,San Bernardino,Apple Valley,1988,NaN,46079,54239,69135


In [20]:
city_array = bigger_ca_cities_df['City']

In [21]:
#import dataframe containing city coordinates
city_latlng = pd.read_csv('cal_cities_lat_long.csv')

In [22]:
#rename columns
city_latlng = city_latlng.rename(columns={'Name': 'City', 'Latitude': 'lat', 'Longitude': 'lng'})

city_latlng.head()

,City,lat,lng
0,Adelanto,34.582769,-117.409214
1,Agoura Hills,34.153339,-118.761675
2,Alameda,37.765206,-122.241636
3,Albany,37.886869,-122.297747
4,Alhambra,34.095286,-118.127014


In [23]:
#merge columns together, so have city populations, and long/lat
city_df_merged = bigger_ca_cities_df.merge(city_latlng, how='inner', on='City')

In [24]:
#
city_df_merged.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010,lat,lng
0,Alameda,Alameda,1854,63852.0,76459,72259,73812,37.765206,-122.241636
1,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089,34.095286,-118.127014
2,Orange,Anaheim,1876,219494.0,266406,328014,336265,33.835292,-117.914503
3,Contra Costa,Antioch,1872,42683.0,62195,90532,102372,38.004922,-121.805789
4,Los Angeles,Arcadia,1903,45993.0,48290,53054,56364,34.139728,-118.035344


In [25]:
final_city_df = city_df_merged[['City', 'pop_april_2010', 'lat', 'lng']]

In [26]:
final_city_df.count()

City              161
pop_april_2010    161
lat               161
lng               161
dtype: int64

In [27]:
final_city_df = final_city_df.rename(columns={'City':'city'})

In [28]:
final_city_df

,city,pop_april_2010,lat,lng
0,Alameda,73812,37.765206,-122.241636
1,Alhambra,83089,34.095286,-118.127014
2,Anaheim,336265,33.835292,-117.914503
3,Antioch,102372,38.004922,-121.805789
4,Arcadia,56364,34.139728,-118.035344
...,...,...,...,...
156,Whittier,85331,33.979178,-118.032844
157,Woodland,55468,38.678517,-121.773297
158,Yorba Linda,64234,33.888625,-117.813111
159,Yuba City,64925,39.140447,-121.616911


In [36]:
lat = "37.765206"
lng = "-122.241636"

loc = f"{lat}, {lng}, 10km"

In [37]:
print(loc)

37.765206, -122.241636, 10km


In [38]:
df_test = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    '@McDonalds geocode:"{}"'.format(loc)).get_items(), 50))[['date', 'content']]

In [39]:
df_test

,date,content
0,2021-07-13 02:12:22+00:00,Randomly started drawing fries and added my fa...
1,2021-07-11 23:24:28+00:00,@McDonalds you have to do better https://t.co/...
2,2021-06-27 10:19:13+00:00,@SmilingBoy123 @McDonalds @michilumin @JRossNi...
3,2021-06-27 05:10:45+00:00,@McDonalds \n#Doge4McDonalds https://t.co/DcuR...
4,2021-06-18 21:20:26+00:00,"Thank you @mcdonalds 🤝 @ Oakland, California h..."
5,2021-06-16 04:03:30+00:00,@McDonalds @teenagedartbag I'm out here checki...
6,2021-06-16 04:02:56+00:00,@McDonalds It's out of stock at every single B...
7,2021-06-15 15:42:57+00:00,@TheShadeRoom wasn’t no @McDonalds in the fron...
8,2021-05-29 02:15:07+00:00,Today I went out of my way to @McDonalds to su...
9,2021-05-26 21:12:55+00:00,@RanDUMBlOve @McDonalds Omg boooo https://t.co...
